In [ ]:
####################################################################
# Machine Learning Primer - Workshop
# Day 2 - September 2021
####################################################################

from __future__ import annotations

# python package imports
from typing import Callable, List, Optional, Tuple

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
####################################################################
# Functions to generate (fake) data that we would use for supervised
# logistic regression (classification)
####################################################################

def line_hypothesis(x: np.ndarray, m: float, c: float) -> np.array:
    return (m * x) + c

def generate_fake_single_var_data(
    n_points: int,
    decision_boundary_true_m: float,
    decision_boundary_true_c: float,
    points_spread: float = 5,
    margin: float = 0,
) -> Tuple[np.ndarray, np.ndarray]:
    """Generate fake positive negative points on both sides of
    the hypothetical decision boundary.
    
    Args:
        n_points: number of sample data points (x, y) to generate.
            Half of the would be positive, half would be negative.
        decision_boundary_true_m: the gradient to use for the
            hypothetical decision boundary.
        decision_boundary_true_c: the intercept to use for the
            hypothetical decision boundary.
        points_spread: The large this number the more the points
            are spread out.
        margin: If positive, points would be pushed away from the
            decision boundary by this amount. If negative, there
            would intermingling of negative / positive points
            (hence making the learning problem harder).
    
    Returns:
        pstv_pnts: Matrix of size (n_points // 2, 2). The first column
            are x values of positive points, and second column are y
            values of positive points.
        ngtv_pnts: Matrix of size (n_points // 2, 2). The first column
            are x values of negative points, and second column are y
            values of negative points.
    """
    n_pstv_points = n_points // 2
    n_ngtv_points = n_points - n_pstv_points

    m, c = decision_boundary_true_m, decision_boundary_true_c
    line_center_x = -(m * c) / (1 + m**2)
    line_center_y = c / (1 + m**2)
    
    # get the unit length path orthogonal to the decision boundary
    unit_ortho_dir_x = -m / ((1 + m**2) ** 0.5)
    unit_ortho_dir_y = 1 / ((1 + m**2) ** 0.5)
    
    # get the unit length path parallel to the decision boundary
    unit_prll_dir_x = unit_ortho_dir_y
    unit_prll_dir_y = -unit_ortho_dir_x
    
    # generate positive points
    # first randomly sample where each point would mobe parallel and then orthogonal
    # to the decision boundary, relative to the center point (line_center_x, line_center_y)
    alpha_prll = np.random.uniform(low=-points_spread, high=points_spread, size=n_pstv_points)
    alpha_ortho = np.random.uniform(low=margin, high=margin + points_spread, size=n_pstv_points)
    
    pstv_pnts = [
        line_center_x + (alpha_prll * unit_prll_dir_x) + (alpha_ortho * unit_ortho_dir_x),
        line_center_y + (alpha_prll * unit_prll_dir_y) + (alpha_ortho * unit_ortho_dir_y),
    ]
    
    # generate negative points (move othogonal to the decision boundary in the opposite dir.)
    # first randomly sample where each point would mobe parallel and then orthogonal
    # to the decision boundary, relative to the center point (line_center_x, line_center_y)
    alpha_prll = np.random.uniform(low=-points_spread, high=points_spread, size=n_ngtv_points)
    alpha_ortho = np.random.uniform(low=-(margin + points_spread), high=-margin, size=n_ngtv_points)
    
    ngtv_pnts = [
        line_center_x + (alpha_prll * unit_prll_dir_x) + (alpha_ortho * unit_ortho_dir_x),
        line_center_y + (alpha_prll * unit_prll_dir_y) + (alpha_ortho * unit_ortho_dir_y),
    ]

    pstv_pnts = np.stack(pstv_pnts, axis=-1)
    ngtv_pnts = np.stack(ngtv_pnts, axis=-1)

    return pstv_pnts, ngtv_pnts

In [ ]:
####################################################################
# Data plotting functionality
####################################################################

def get_hypothesis_scatter_plots(
    pstv_pnts: np.ndarray,
    ngtv_pnts: np.ndarray,
    hypothesis_params: Optional[np.ndarray] = None,
) -> List[go.Scatter]:
    """Gets the scatter plots to draw the data, true line, and hypothesis
    line if parameters given.
    
    Args:
        pstv_data: (N, C) array of positive data points.
        pstv_data: (M, C) array of negative data points.
        hypothesis_params: (C) vector of current parameters. If given, will
            be used to draw the hypothesis line.
    
    Returns:
        List of scatter plots for drawing data, true line, and hypothesis.
    """    
    min_x = min(np.min(pstv_pnts[:, 0]), np.min(ngtv_pnts[:, 0])) - 1
    max_x = max(np.max(pstv_pnts[:, 0]), np.max(ngtv_pnts[:, 0])) + 1
    
    scatter_plots = [
        # plot the positive data points
        go.Scatter(
            x=pstv_pnts[:, 0],
            y=pstv_pnts[:, 1],
            mode="markers",
            marker_size=10,
            marker_symbol='circle',
            marker_color='blue',
            name="Positive",
        ),
        # plot the negative data points
        go.Scatter(
            x=ngtv_pnts[:, 0],
            y=ngtv_pnts[:, 1],
            mode="markers",
            marker_size=10,
            marker_symbol='x',
            marker_color='red',
            name="Negative",
        ),
        # plot the underlying GT line
        go.Scatter(
            x=[min_x, max_x],
            y=line_hypothesis(
                np.array([min_x, max_x]),
                m=decision_boundary_true_m,
                c=decision_boundary_true_c
            ),
            mode='lines',
            line_dash='dash',
            line_color='green',
            name='GT decision bndry'
        )
    ]
    if hypothesis_params is not None:
        # a + bx + cy = 0  ->  y = -b/c x - a/c
        hypothesis_m = -hypothesis_params[1] / hypothesis_params[2]
        hypothesis_c = -hypothesis_params[0] / hypothesis_params[2]
        # plot the current hypothesis
        scatter_plots.append(
            go.Scatter(
                x=[min_x, max_x],
                y=line_hypothesis(
                    np.array([min_x, max_x]),
                    m=hypothesis_m,
                    c=hypothesis_c
                ),
                mode='lines',
                # line_dash='dash',
                line_color='purple',
                name='hypothesis'
            )
        )
    return scatter_plots


def plot_gradient_descent_info(
    pstv_data: np.ndarray,
    ngtv_data: np.ndarray,
    params_history: np.ndarray,
):
    """Draws data/hypothesis; cost contour map; and cost vs training iterations.
        
    Args:
        pstv_data: (N, C) array of positive data points.
        pstv_data: (M, C) array of negative data points.
        params_history: (T, C) array giving all the parameters as the training
            progresses (from the oldest to the newest).
    """
    common_axis = dict(
        mirror=True,
        ticks='outside',
        showline=True,
        linewidth=2,
        linecolor='black'
    )
    
    fig = make_subplots(
        rows=2,
        cols=2,
        specs=[[{}, {}], [{"colspan": 2}, None]],
        row_heights=[0.7, 0.3],
        # horizontal_spacing=0.01,
        vertical_spacing=0.2,
        subplot_titles=("Hypothesis", "Cost map", "Cost w/ iteration")
    )
    
    # plot the data, the hypothesis, and true line
    scatter_plots = get_hypothesis_scatter_plots(
        pstv_pnts=pstv_data[:, 1:],
        ngtv_pnts=ngtv_data[:, 1:],
        hypothesis_params=params_history[-1],
    )
    for plot in scatter_plots:
        fig.add_trace(plot, row=1, col=1)
    fig.update_xaxes(title_text="Feature 1 (-b/c)", row=1, col=1, **common_axis)
    fig.update_yaxes(title_text="Feature 2 (-a/c)", row=1, col=1, **common_axis)
    
    
    # the range of gradient/intercept values
    grad_plot_range = np.linspace(-10, 10, 101)
    intcpt_plot_range = np.linspace(-10, 10, 101)

    # a + bx + cy = 0  ->  y = -b/c x - a/c
    c = params_history[-1][2]
    a_plot_range = -intcpt_plot_range * c
    b_plot_range = -grad_plot_range * c
    
    # get all the [a, b, c] parameters for the contour map grid
    grid_a, grid_b = np.meshgrid(a_plot_range, b_plot_range)
    grid_a = np.reshape(grid_a, (-1))
    grid_b = np.reshape(grid_b, (-1))
    grid_params = np.stack([grid_a, grid_b, np.full_like(grid_a, c)], axis=-1)
    
    # compute the cost at each parameter set
    grid_costs = compute_cost(
        pstv_data=pstv_data, ngtv_data=ngtv_data, params=grid_params.T
    )
    grid_costs = np.reshape(grid_costs, (b_plot_range.size, a_plot_range.size))
    
    # plot the cost contour map against m and c parameters
    fig.add_trace(
        go.Contour(x=grad_plot_range, y=intcpt_plot_range, z=grid_costs.T),
        row=1, col=2
    )
    
    # convert [a,b,c] parameter values into gradient / intercept
    # a + bx + cy = 0  ->  y = -b/c x - a/c
    x_history = -params_history[:,1] / params_history[:,2]
    y_history = -params_history[:,0] / params_history[:,2]
    # plot the history of gradient/intercept values on the cost contour map
    fig.add_trace(
        go.Scatter(
            x=x_history,
            y=y_history,
            mode="markers+lines",
            marker_size=[5]*(len(x_history)-1) + [10],
            marker_color='white',
            name="params history"
        ),
        row=1, col=2,
    )
    # draw a marker for true gradient/intercept 
    fig.add_trace(
        go.Scatter(
            x=[decision_boundary_true_m],
            y=[decision_boundary_true_c],
            mode="markers",
            marker_size=10,
            marker_color='yellow',
            marker_symbol='x',
            name="True params"
        ),
        row=1, col=2,
    )
    # text annotation giving the current c value
    fig.add_annotation(
        text=f"Current c = {c:.6f}",
        x=np.min(grad_plot_range),
        y=np.max(intcpt_plot_range),
        font=dict(color="black", size=14),
        bgcolor="white",
        showarrow=False,
        xanchor="left",
        yanchor="top",
        row=1,
        col=2,
    )
    fig.update_xaxes(
        title_text="-b/c (gradient)",
        range=[np.min(grad_plot_range), np.max(grad_plot_range)],
        row=1, col=2, **common_axis
    )
    fig.update_yaxes(
        title_text="-a/c (intercept)",
        range=[np.min(intcpt_plot_range), np.max(intcpt_plot_range)],
        row=1, col=2, **common_axis
    )
    
    # plot history of the cost against training iterations
    cost_history = compute_cost(
        pstv_data=pstv_data, ngtv_data=ngtv_data, params=params_history.T
    )
    fig.add_trace(
        go.Scatter(
            x=list(range(1, len(cost_history) + 1)),
            y=cost_history,
            mode="markers+lines",
            name="cost",
        ),
        row=2, col=1,
    )
    fig.update_xaxes(title_text="Step", row=2, col=1, **common_axis)
    fig.update_yaxes(title_text="Cost", row=2, col=1, **common_axis)
    
    # move the legend
    fig.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.05,
            xanchor="right",
            x=1
        )
    )
    
    fig.show()

In [ ]:
####################################################################
# Generate some (fake) data to do supervised classification
####################################################################

np.random.seed(1)

N = 100
data_spread = 2
margin = 0

decision_boundary_true_m = 2   # np.random.uniform(low=-5, high=5)
decision_boundary_true_c = -3  # np.random.uniform(low=-5, high=5)

pstv_pnts, ngtv_pnts = generate_fake_single_var_data(
    n_points=N,
    decision_boundary_true_m=decision_boundary_true_m,
    decision_boundary_true_c=decision_boundary_true_c,
    points_spread=data_spread,
    margin=margin,
)

# add a column of 1s to make all the subsequent computation easier
pstv_data = np.concatenate([np.ones((pstv_pnts.shape[0], 1)), pstv_pnts], axis=-1)
ngtv_data = np.concatenate([np.ones((ngtv_pnts.shape[0], 1)), ngtv_pnts], axis=-1)

####################################################################
# Plot the generated data (and the underlying decision boundary)
####################################################################

fig = go.Figure(
    layout=dict(
        xaxis_title="Feature 1 (-b/c)",
        yaxis_title="Feature 2 (-a/c)",
        title=(
            "True decision boundary: "
            f"gradient: {decision_boundary_true_m}, "
            f"intercept: {decision_boundary_true_c}"
        ),
        title_x=0.5,
        width=630,
        height=500,
        xaxis_range=(-4, 4),
        yaxis_range=(-4, 4),
    )
)

scatter_plots = get_hypothesis_scatter_plots(
    pstv_pnts=pstv_pnts,
    ngtv_pnts=ngtv_pnts,
)
for plot in scatter_plots:
    fig.add_trace(plot)

fig.show()   

In [ ]:
def compute_cost(
    pstv_data: np.ndarray,
    ngtv_data: np.ndarray,
    params: np.ndarray,
) -> float | np.ndarray:
    """Compute the cost at specified parameter values.
    
    Args:
        pstv_data: (N, C) array of positive data points.
        pstv_data: (M, C) array of negative data points.
        params: (C) vector of current parameters. OR (C, P) array which
            is a set of P different parameters.
    
    Returns:
        Cost scalar value, OR if `params` is a (C, P) array then it
        would be a P sized vector of costs.
    """
    
    ############################################
    # IMPLEMENT THIS
    ############################################
    if params.ndim == 1:
        return 0
    else:
        return np.full(fill_value=0, shape=params.shape[1])


def gradient_params(
    pstv_data: np.ndarray,
    ngtv_data: np.ndarray,
    params: np.ndarray,
) -> np.ndarray:
    """Compute the gradient of the cost with respect to parameters
    
    Args:
        pstv_data: (N, C) array of positive data points.
        pstv_data: (M, C) array of negative data points.
        params: (C) vector of current parameters.
    
    Returns:
        Gradient of cost with respect to all different parameters. This
        would be an vector of size (C).
    """
    
    ############################################
    # IMPLEMENT THIS
    ############################################
    return np.zeros_like(params)

    
def gradient_descent_step(
    pstv_data: np.ndarray,
    ngtv_data: np.ndarray,
    curr_params: np.ndarray,
    learning_rate: float
) -> Tuple[float, float]:
    """Runs a single step of gradient descent
    
    Args:
        pstv_data: (N, C) array of positive data points.
        pstv_data: (M, C) array of negative data points.
        curr_params: (C) vector of current parameters.
        learning_rate: Step size for gradient descent.
    
    Returns:
        (C) vector of updated parameters after gradient descent.
    """
    
    ############################################
    # IMPLEMENT THIS
    ############################################
    return curr_params.copy()

In [ ]:
####################################################################
# Run gradient descent for n_steps
####################################################################

n_steps = 1

# some initial hypothesis (can be any random value)
hypothesis_params = np.array([5, 5, 1])

# the step size used during gradient descent
learning_rate = 1e0

# used for plotting the progress of the parameters
params_history = [hypothesis_params]

for idx in range(n_steps):
    # compute the current cost (loss)
    cost = compute_cost(
        pstv_data=pstv_data,
        ngtv_data=ngtv_data,
        params=hypothesis_params,
    )
    
    # get the new slope / intercept by gradient descent
    hypothesis_params = gradient_descent_step(
        pstv_data=pstv_data,
        ngtv_data=ngtv_data,
        curr_params=hypothesis_params,
        learning_rate=learning_rate
    )
    
    # a + bx + cy = 0  ->  y = -b/c x - a/c
    hypothesis_m = -hypothesis_params[1] / hypothesis_params[2]
    hypothesis_c = -hypothesis_params[0] / hypothesis_params[2]
    print(f"Step {idx: <3},  Cost: {cost:.5f},  {hypothesis_params} = ({hypothesis_m}, {hypothesis_c})")

    params_history.append(hypothesis_params)

# plot the whole gradient descent process
fig = plot_gradient_descent_info(
    pstv_data=pstv_data,
    ngtv_data=ngtv_data,
    params_history=np.stack(params_history, axis=0),
)